# 모델링 구현

## 순환 신경망 분류 모델

- 딥러닝을 활용해 분류하는 모델

- 문장 데이터를 입력해서 문장 흐름에서 패턴을 찾아 분류

- 주어진 단어 특징 벡터를 활용해 모델을 학습하지 않고 텍스트 정보를 입력해서 문장에 대한 특징 정보를 추출

### 순환 신경망(RNN)
- 현재 정보는 이전 정보가 점층적으로 쌓이면서 정보를 표현할 수 있는 모델
- 시간에 의존적인 또는 순차적인 데이터에 대한 문제에 주로 활용됨



- 입력상태(input state)
- 은닉 상태(hidden state)

### 학습 데이터 불러오기

In [3]:
import numpy as np
import json

- 전처리한 데이터 불러오기

In [8]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

INPUT_TRAIN_DATA_FILE_NAME = 'train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'train_label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

# 입력 데이터
input_data = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
# 정답 라벨 데이터
label_data = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))

prepro_configs = None

with open(DATA_IN_PATH + DATA_CONFIGS_FILE_NAME, 'r') as f:
    prepro_configs = json.load(f)

### 학습과 검증 데이터셋 분리

In [9]:
# 학습과 검증 데이터셋 분리하기
from sklearn.model_selection import train_test_split

# 책에 주어진 임의의 숫자 사용함
TEST_SPLIT = 0.1
RANDOM_SEED = 13371447

train_input, test_input, train_label, test_label = train_test_split(input_data, label_data, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [17]:
print(train_input.shape)
print(test_input.shape)
print(train_label.shape)
print(test_label.shape)

(22500, 174)
(2500, 174)
(22500,)
(2500,)


### 데이터 입력 함수

In [19]:
# 데이터 입력 함수
import tensorflow as tf

BATCH_SIZE = 16
NUM_EPOCHS = 3

def mapping_fn(X, Y):
    inputs, labels = {'x' : X}, Y

    return inputs, labels


def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    iterator = dataset.make_one_shot_iterator()

    return iterator.get_next()


def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_input, test_label))
    dataset = dataset.map(mapping_fn)
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_one_shot_iterator()

    return iterator.get_next()

#### tf.data.Dataset.from_tensor_slices
- 주어진 데이터들을 묶어서 조각으로 만들고 함께 사용할 수 있게 해준다.

- make_one_shot_iterator()
    - 데이터를 하나씩 사용할 수 있게 만들어준다.

- iterator의 get_next() 함수를 사용하여 데이터가 하나씩 나오게 되는 구조

- dataset.batch()
    - 배치 크기 설정

- dataset.shuffle()
    - 데이터를 셔플

- 에폭(Epoch)
    - 전체 데이터를 몇 번 사용하는지를 지칭하는 말

- dataset.repeat()
    - 전체 데이터 반복 설정

- dataset.map()
    - 매핑 기능
    - 라벨을 제외한 나머지 데이터를 하나의 입력값으로 묶기 위한 과정

- 데이터 입력 함수는 모델에 학습 데이터를 주입하는 방법


### 모델 함수